# Phase 4 Code Challenge Review

## Overview

- Pipelines and gridsearching
- Ensemble Methods
- Natural Language Processing
- Clustering

In [92]:
# Basic Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline

In [93]:
# from src.call import call_on_students

# 1) Pipelines and Gridsearching

### What are the benefits of using a pipline?

In [94]:
# call_on_students(1)

Create a pipeline →  it becomes an sklearn estimator →  fit, transform, cross validate, etc. 
- Compartmentalizes each part of the process
- Flexible - can change each part separately
- Helps prevent data leakage in cross validation:
- Column transformer: calling diff transformation objects on diff columns. Ohe, scaling, imputation only on the columns that need it. Like “mini” pipelines for each step
- Handling Missing Values (nulls) can happen differently on diff columns


### What does a gridsearch achieve?

In [95]:
# call_on_students(1)

The problem with doing model tuning by hand - sometimes each adjustment would affect the other if you combined adjustments
- Really you need to tune the model is tandem

Grid Search:
Automatically/Systematically searching for the optimal combination of hypterparametres by providing a grid to our Grid Search with the values of each hypterparameter that we want to try

Grid search will evaluate all those combos with the evaluation metric that we choose, then return the best combination of hypterparameters = the best model

scikit-Learn’s GridSearchCV class w/ .fit() method 
Cross validates each of those combinations


### Set up a pipeline with a scaler and a logistic regression model on the breast cancer dataset that predicts whether the tumor is malignant (target = 1). Don't worry for now about a train-test split.

In [96]:
# call_on_students(1)

**Answer**:

In [97]:
from sklearn.datasets import load_breast_cancer
X, y = load_breast_cancer(return_X_y=True)

In [98]:
# Your code here
pipeline = Pipeline([('scaler', StandardScaler()),
                   ('lr', LogisticRegression(random_state=1, C=1))])

In [99]:
# call_on_students(1)

### Split the data into train and test and then gridsearch over pipelines like the one you just built to find the best-performing model. Try C (inverse regularization) values of 10, 1, and 0.1. Try out the best estimator on the test set.

In [100]:
# call_on_students(1)

**Answer**:

In [101]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [102]:
# Your code here
grid = {
    'lr__C': [0.1, 1, 10],
}

grid_search = GridSearchCV(pipeline, grid, verbose=2, return_train_score=True)
#default cv=5

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] lr__C=0.1 .......................................................
[CV] ........................................ lr__C=0.1, total=   0.0s
[CV] lr__C=0.1 .......................................................
[CV] ........................................ lr__C=0.1, total=   0.0s
[CV] lr__C=0.1 .......................................................
[CV] ........................................ lr__C=0.1, total=   0.0s
[CV] lr__C=0.1 .......................................................
[CV] ........................................ lr__C=0.1, total=   0.0s
[CV] lr__C=0.1 .......................................................
[CV] ........................................ lr__C=0.1, total=   0.0s
[CV] lr__C=1 .........................................................
[CV] .......................................... lr__C=1, total=   0.0s
[CV] lr__C=1 .........................................................
[CV] ............

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ......................................... lr__C=10, total=   0.0s


[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.2s finished


GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('lr',
                                        LogisticRegression(C=1,
                                                           random_state=1))]),
             param_grid={'lr__C': [0.1, 1, 10]}, return_train_score=True,
             verbose=2)

In [103]:
grid_search.cv_results_

{'mean_fit_time': array([0.00904102, 0.00924492, 0.01708112]),
 'std_fit_time': array([0.00586284, 0.000918  , 0.00178607]),
 'mean_score_time': array([0.00047827, 0.00034547, 0.00042601]),
 'std_score_time': array([1.69403875e-04, 2.55644314e-05, 1.28482921e-04]),
 'param_lr__C': masked_array(data=[0.1, 1, 10],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'lr__C': 0.1}, {'lr__C': 1}, {'lr__C': 10}],
 'split0_test_score': array([0.95348837, 0.95348837, 0.94186047]),
 'split1_test_score': array([0.95294118, 0.95294118, 0.94117647]),
 'split2_test_score': array([0.97647059, 0.97647059, 1.        ]),
 'split3_test_score': array([0.98823529, 0.98823529, 0.96470588]),
 'split4_test_score': array([0.98823529, 0.98823529, 0.97647059]),
 'mean_test_score': array([0.97187415, 0.97187415, 0.96484268]),
 'std_test_score': array([0.01583032, 0.01583032, 0.02217898]),
 'rank_test_score': array([1, 1, 3], dtype=int32),
 'split0_train_scor

^ in the above we have 5 splits (splits 0 through 4 in rows) with each one containing 3 parameters

In [104]:
grid_search.best_params_

{'lr__C': 0.1}

In [105]:
grid_search.best_estimator_
# could save this off as final_model

Pipeline(steps=[('scaler', StandardScaler()),
                ('lr', LogisticRegression(C=0.1, random_state=1))])

In [106]:
grid_search.best_score_

0.97187414500684

In [107]:
#testing the best estimator out on test data
grid_search.best_estimator_.score(X_test, y_test)

0.958041958041958

# 2) Ensemble Methods

### What sorts of ensembling methods have we looked at?

In [108]:
# call_on_students(1)

Ensemble: a model consisting of a bunch of other models
We want differently randomized models that are overfit in different ways

PROS
- Generalize (perform on unseen test data) well
- Decreases variance →  less overfitting
- More complexity: have to train each model or part of model

CONS:
- BUT sometimes means →  less interpretation
    - Ensembles consisting of a bunch of trees still have feature importances
    - But ensembles consisting of trees and logistic regs and knn’s have feature imps, coeffs, and	but not those for the whole model
- Takes up more space: have to keep each model
- Also means →  more computational power and time

FULL LIST
1. Random forest
2. Gradient Boosting:
    - XG boost
3. Adaboost
4. Bagging
5. Stacking
6. Voting
7. Extra trees
 
Bagging: a bag of marbles, each marble is a model
- Many model types naturally overfit
- So, we can randomly assign data to each model
- Each model overfits in different ways
- Then we aggregate all the models 
- Bootstrap (replacement) aggregating
- Repeat algorithm many times w/ replacement (bootstrap)
- Create multiple samples from your data
- Train models on those samples
- Final model predicts by averaging or voting across those models
    - →  hope that that smooths over/cancels out the different ways each model overfits to reduce variance
    - →  low variance since it averages out quirks the individual models might’ve learned
Bagging:
- Because we're resampling our data with replacement, we're bootstrapping.
- Because we're making use of our many samples' predictions, we're aggregating.
- Because we're bootstrapping and aggregating all in the same algorithm, we're bagging.

4 Levels of Randomization
1. Simple Bag: BaggingClassifier
- Train each model on random samples w/ replacement
- Bagging Algorithm
    1. Take a sample of your X_train and fit a decision tree to it.
    2. Replace the first batch of data and repeat.
    3. When you've got as many trees as you like, make use of all your individual trees' predictions to come up with some holistic prediction. (Most obviously, we could take the average of our predictions, but there are other methods we might try.)

2. Random Forest: random sample AND give model all the features, but at each node only allow it to choose from a random subset of those features to split on, using a splitting criteria like gini or entropy to split on
    - Each tree in the forest is very different
    - Finds best split based on gini or entropy, like normal decision trees
3. Extra Trees: random sample, AND random subset of features, AND among that random subset of features, choosing a random value for each feature and calculating gini or entropy for those values. then whichever value is best, split on that feature

Ensembles of Different Model Types: 
Maybe you have some linear data, some non-linear, a lot of categorical data
Logreg models better for linear relationships, but trees are better for the last two
2 methods: 
1. VotingClassifier or VotingRegressor: taking a simple average (or a vote) of the outputs of the sub-models
Sometimes uses weighted averaging
2. StackingClassifier or StackingRegressor: feeding the ouputted predictions of a bunch of sub-models as features in another final model
    - Also known as Meta-Classifier/Meta-Regressor
    - Approaching a neural network 
    - Estimates the weights in the averaging for us
    - This means we'll have one layer of base estimators and another layer that is "trained to optimally combine the model predictions to form a new set of predictions". See this short blog post for more.


### What is random about a random forest?

In [109]:
# call_on_students(1)

-Still bootstrapping: random samples w/ replacement
- Now give each tree all the features, but at each node we give the tree a random subset of the features to choose from/split on 

Pros:

- High performance, low variance
- Transparent: inherited from Decision Tree (white box??)

Cons:

- So many trees to plant
- Computationally expensive
- Memory: every tree stored back to memory, almost as memory intensive as KNN


### What hyperparameters of a random forest might it be useful to tune? How so?

In [110]:
# call_on_students(1)

- n_estimators: #s of sub-models to train
- Max_features: feature subsetting
    - Can also set this parameter in a Bagging Classifier and it will make that bagging ~ Random Forest
- Bootstrap: whether you want to random sample the dataset or not. We do want this in order to grow different trees each time
- Max_samples: size of the random samples going into each model. controls  
    - Must be 0 -1
    - Default = None, will sample the length of X
    - If it is = 1: →  
    - This can be the same size as the full datase t b/c we allow replacement after we take each data point
    - So we can have a bunch of diff samples that are all the same size as the original for each model
- also all same parameters as a decision tree: max depth, min samples leaf, min samples split, criterion

### Build a random forest model on the breast cancer dataset that predicts whether the tumor is malignant (target = 1).

In [111]:
# call_on_students(1)

**Answer**:

In [112]:
# Your code here
rfc = RandomForestClassifier(random_state=1)
rfc.fit(X_train, y_train)
# y_pred = rfc.predict(X_test)
# ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred), display_labels='0 class', '1 class').plot()

# from sklearn.model_selection import cross_val_score
# scores = cross_val_score(estimator=rfc, X=X_train, y=y_train, cv=5)
# scores

print(rfc.score(X_train, y_train)) # train accuracy
print(rfc.score(X_test, y_test)) # test accuracy


1.0
0.9440559440559441


^ this is expected b/c tree based models tend to be overfit!

# 3) Natural Language Processing

## NLP Concepts

### Some Example Text

In [113]:
# Each sentence is a document
sentence_one = "Harry Potter is the best young adult book about wizards"
sentence_two = "Um, EXCUSE ME! Ever heard of Earth Sea?"
sentence_three = "I only like to read non-fiction.  It makes me a better person."

# The corpus is composed of all of the documents
corpus = [sentence_one, sentence_two, sentence_three]

### NLP Pre-processing

List at least three steps you can take to turn raw text like this into something that would be semantically valuable (aka ready to turn into numbers):

In [114]:
# call_on_students(1)

1. Remove stop words like "is" that have no semantic value
2. Lower case all the words
3. Remove punctuation
4. Stemming or lemmatize

### Describe what vectorized text would look like as a dataframe.

If you vectorize the above corpus, what would the rows and columns be in the resulting dataframe (aka document term matrix)

In [115]:
# call_on_students(1)

Represented as a document term matrix:
- Each row is a document in our corpus
- Each column is a unique vocabulary n-gram (a token). 1 or 0 for if that doc has it
- the values can be a count, presence, or some form of score (like tf-idf
- typically returned as a sparse matrix given lots of 0 values
- So each row is a numerical vector


### What does TF-IDF do?

Also, what does TF-IDF stand for?

In [116]:
# call_on_students(1)

Tf-Idf: Probably the most commonly used. Very powerful for content-based classification b/c adds importance weight to certain tokens
Useful when the goal is to distinguish the content of documents from others in the corpus.

TF-IDF score: higher = the more important that word is in that document compared to how important it is in all the documents 

TfidVectorizer will weight words by how important it is to that doc
So the sort_values of each type of vectorizer in each doc will look different
- Tf = term frequency (frequency of that word in that document)
- idf = inverse document frequency (log calc of frequency of that word across the full corpus)


## NLP in Code

### Set Up

In [117]:
# New section, new data
policies = pd.read_csv('data/2020_policies_feb_24.csv')

def warren_not_warren(label):
    
    '''Make label a binary between Elizabeth Warren
    speeches and speeches from all other candidates'''
    
    if label =='warren':
        return 1
    else:
        return 0
    
policies['candidate'] = policies['candidate'].apply(warren_not_warren)

The dataframe loaded above consists of policies of 2020 Democratic presidential hopefuls. The `policy` column holds text describing the policies themselves.  The `candidate` column indicates whether it was or was not an Elizabeth Warren policy.

In [118]:
policies.head()

,Unnamed: 0,name,policy,candidate
0,0,100% Clean Energy for America,"As published on Medium on September 3rd, 2019:...",1
1,1,A Comprehensive Agenda to Boost America’s Smal...,Small businesses are the heart of our economy....,1
2,2,A Fair and Welcoming Immigration System,"As published on Medium on July 11th, 2019:\nIm...",1
3,3,A Fair Workweek for America’s Part-Time Workers,Working families all across the country are ge...,1
4,4,A Great Public School Education for Every Student,I attended public school growing up in Oklahom...,1


The documents for activity are in the `policy` column, and the target is candidate. 

### Import the Relevant Class, Then Instantiate and Fit a Count Vectorizer Object

In [119]:
# call_on_students(1)

In [120]:
policies.candidate.value_counts()

0    114
1     75
Name: candidate, dtype: int64

In [121]:
# First! Train-test split the dataset
X = policies.policy
y= policies.candidate
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [122]:
# Import the relevant vectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [123]:
# Instantiate it
vec = CountVectorizer(stop_words='english', strip_accents='unicode')
# could enter stop words here

In [124]:
# Fit it
vec.fit(X_train)

CountVectorizer(stop_words='english', strip_accents='unicode')

### Vectorize Your Text, Then Model

In [ ]:
# call_on_students(1)

In [125]:
# Code here to transform train and test sets with the vectorizer
X_train_vectorized = vec.transform(X_train)
X_test_vectorized = vec.transform(X_test)

In [126]:
# Code here to instantiate and fit a Random Forest model
rfc2 = RandomForestClassifier(random_state=1)
rfc2.fit(X_train_vectorized, y_train)

RandomForestClassifier(random_state=1)

In [38]:
# Code here to evaluate your model on the test set
y_pred2 = rfc2.predict(X_test)
ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred2)).plot()

# ^ why doesn't this work?

ValueError: could not convert string to float: "In 1987, the United Church of Christ's Commission on Racial Justice commissioned one of the first studies on hazardous waste in communities of color. A few years later -  28 years ago this month - delegates to the First National People of Color Environmental Leadership Summit adopted 17 principles of environmental justice. But in the years since, the federal government has largely failed to live up to the vision these trailblazing leaders outlined, and to its responsibilities to the communities they represent. \nFrom predominantly black neighborhoods in Detroit to Navajo communities in the southwest to Louisiana’s Cancer Alley, industrial pollution has been concentrated in low-income communities for decades - communities that the federal government has tacitly written off as so-called “sacrifice zones.” But it’s not just about poverty, it’s also about race. A seminal study found that black families are more likely to live in neighborhoods

In [35]:
scores = cross_val_score(estimator=rfc2, X=X_train_vectorized, y=y_train, cv=5)
scores


NameError: name 'cross_val_score' is not defined

In [40]:
print(rfc2.score(X_train_vectorized, y_train)) # train accuracy
print(rfc2.score(X_test_vectorized, y_test)) # test accuracy

1.0
0.9375


Is very overfit, how to address this? We let the count vectorizer run unchecked on all of our features in the train data --> too much complexity --> overfitting

We should tune the countvectorizer, add a max_features parameter

In [ ]:
cv = CountVectorizer(stop_words='english', strip_accents-'unicode'

# 4) Clustering

## Clustering Concepts

### Describe how the K-Means algorithm updates its cluster centers (centroids) after initialization.

In [ ]:
# call_on_students(1)

Process: animation in the nb under methods. 4 clusters set as an example
1. Randomly choose 4 “centroids” = data points that we randomly decide to become our “center point”
    - n_clusters parameter =  # of clusters = # of centroids. Fewer = easier to visualize
2. We assign all the other data points in the dataset to their closest centroid, forming n clusters
3. We take the mean of those 4 clusters and move the centroid to the true mean of that cluster 
    - Goal is to get the tightest clusters possible. Minimizing total intra-cluster distance (sum of distance from centroid to cluster data points)
    - Also want to maximize total inter-cluster distance (distance between data points in one cluster and data points in other clusters)
4. Using that new centroid, we repeat from step 2 again
5. Problem: the resultant 4 clusters will look different depending on the 4 random centroids you chose in step 1
    - Solution: do a bunch of iterations of this with a bunch of different random starting points, then the algorithm
    - n_init parameter = the number of times the k-means algorithm is run with different centroid seeds

### What is inertia, and how does K-Means use inertia to determine the best estimator?

Please also describe the method you can use to evaluate clustering using inertia.

Documentation, for reference: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

In [ ]:
# call_on_students(1)

Inertia: squared intra-cluster distance. The sum of squared distances SSE, weighted by the sample weights. Squared = outliers have a big effect
- model.inertia_

Elbow Method:
- Code to plot the SSE’s at various k’s in the nb
- Inertia is on the y axis
- Calculate the inertia for a bunch of different k’s
- Inertia will always decrease as you increase k, b/c no matter what, the more cluster’s you add (k), the tighter those clusters will be (lower intra-cluster distance, smaller inertia)
- Therefore, we are looking for the elbow: whichever k value is associated with the biggest change in inertia. The point of inflection. 


### What other metric do we have to score the clusters which are formed?

Describe the difference between it and inertia.

In [ ]:
# call_on_students(1)

Silhouette Coefficient: more interpretable than elbow, b/c sometimes elbow can have multiple “elbows”/bends
aklearn.metrics.silhouette_score
- Calculated using:
- a = another measure of intra-cluster distance, diff from inertia though. Not the sum of squared, just the avg distance between a point and all the other points in that cluster
- b = INTER-cluster distance. Trying to maximize
- Silhouette coefficient ranges from -1 to 1
    - Closer to 1 = more clearly defined clusters
    - Closer to -1 = data points are incorrectly assigned
- Calculate silhouette score for each data point, then average them
- We just pick the k with the highest silhouette coefficient (peak in the graph below)


## Clustering in Code with Heirarchical Agglomerative Clustering

After the above conceptual review of KMeans, let's practice coding with agglomerative clustering.

### Set Up

In [41]:
# New dataset for this section!
from sklearn.datasets import load_iris

data = load_iris()
X = pd.DataFrame(data['data'])

In [42]:
X

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


### Prepare our Data for Clustering

What steps do we need to take to preprocess our data effectively?


In [ ]:
# call_on_students(1)

Need to standard scale - clustering is distance based

In [54]:
# Code to preprocess the data
scaler = StandardScaler()

# Name the processed data X_processed
X_processed = scaler.fit_transform(X)

In [58]:
X_processed

array([[-9.00681170e-01,  1.01900435e+00, -1.34022653e+00,
        -1.31544430e+00],
       [-1.14301691e+00, -1.31979479e-01, -1.34022653e+00,
        -1.31544430e+00],
       [-1.38535265e+00,  3.28414053e-01, -1.39706395e+00,
        -1.31544430e+00],
       [-1.50652052e+00,  9.82172869e-02, -1.28338910e+00,
        -1.31544430e+00],
       [-1.02184904e+00,  1.24920112e+00, -1.34022653e+00,
        -1.31544430e+00],
       [-5.37177559e-01,  1.93979142e+00, -1.16971425e+00,
        -1.05217993e+00],
       [-1.50652052e+00,  7.88807586e-01, -1.34022653e+00,
        -1.18381211e+00],
       [-1.02184904e+00,  7.88807586e-01, -1.28338910e+00,
        -1.31544430e+00],
       [-1.74885626e+00, -3.62176246e-01, -1.34022653e+00,
        -1.31544430e+00],
       [-1.14301691e+00,  9.82172869e-02, -1.28338910e+00,
        -1.44707648e+00],
       [-5.37177559e-01,  1.47939788e+00, -1.28338910e+00,
        -1.31544430e+00],
       [-1.26418478e+00,  7.88807586e-01, -1.22655167e+00,
      

### Import the Relevant Class, Then Instantiate and Fit a Hierarchical Agglomerative Clustering Object

Let's use `n_clusters = 2` to start (default)

In [ ]:
# call_on_students(1)

In [127]:
# Import the relevent clustering algorithm
from sklearn.cluster import AgglomerativeClustering

# Instantiate and fit
agc = AgglomerativeClustering(n_clusters = 2)
preds = agc.fit_predict(X_processed)

# or just .fit without assigning it to preds, 
# then do preds = agc.labels_ 

In [56]:
# Calculate a silhouette score
from sklearn.metrics import silhouette_score

silhouette_score(X_processed, preds)

0.5770346019475989

### Write a Function to Test Different Options for `n_clusters`

The function should take in the number for `n_clusters` and the data to cluster, fit a new clustering model using that parameter to the data, print the silhouette score, then return the labels attribute from the fit clustering model.

In [57]:
# call_on_students(1)

def cluster_test(n_clusters, X):
    agc = AgglomerativeClustering(n_clusters = n_clusters)
    preds = agc.fit_predict(X)
    return silhouette_score(X, preds), agc.labels_

cluster_test(3, X_processed)

(0.446689041028591,
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 0, 2, 0,
        2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 0, 2,
        2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [130]:
for k in range(2, 10):
    print(cluster_test(k, X_processed))

(0.5770346019475989, array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
(0.446689041028591, array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 0, 2, 0,
       2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 0, 2,
       2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0

In [ ]:
""